# Decision Tree Lab

In [229]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
import numpy as np
from scipy.io import arff
import matplotlib.pyplot as plt
import pandas as pd
import json
from random import seed
from random import randrange

## 1. (40%) Correctly implement the ID3 decision tree algorithm, including the ability to handle unknown attributes (You do not need to handle real valued attributes).  
### Code Requirements/Notes:
- Use standard information gain as your basic attribute evaluation metric.  (Note that normal ID3 would usually augment information gain with gain ratio or some other mechanism to penalize statistically insignificant attribute splits. Otherwise, even with approaches like pruning below, the SSE type of overfit could still hurt us.) 
- You are welcome to create other classes and/or functions in addition to the ones provided below. (e.g. If you build out a tree structure, you might create a node class).
- It is a good idea to use a simple data set (like the lenses data or the pizza homework), which you can check by hand, to test your algorithm to make sure that it is working correctly. 

In [24]:
class DTClassifier(BaseEstimator,ClassifierMixin):

    def __init__(self,counts=None):
        """ Initialize class with chosen hyperparameters.
        Args:
        Optional Args (Args we think will make your life easier):
            counts: A list of Ints that tell you how many types of each feature there are
        Example:
            DT  = DTClassifier()
            or
            DT = DTClassifier(count = [2,3,2,2])
            Dataset = 
            [[0,1,0,0],
            [1,2,1,1],
            [0,1,1,0],
            [1,2,0,1],
            [0,0,1,1]]

        """
        self.ig_splits = []
        

    def fit(self, x, y, Print=False):
        """ Fit the data; Make the Decision tree

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets
            y (array-like): A 1D numpy array with the training targets

        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)

        """
        data = x.copy()
        data[y.name] = y
        self.tree = self.convert(self.decision_tree(data, data, x.columns, y.name, Print))


    def predict(self, X):
        """ Predict all classes for a dataset X

        Args:
            X (array-like): A 2D numpy array with the training data, excluding targets

        Returns:
            array, shape (n_samples,)
                Predicted target values per element in X.
        """
        
        # convert input data into a dictionary of samples
        samples = self.convert(X.to_dict(orient='records'))
        
        predictions = []
        # make a prediction for every sample
        for sample in samples:
          predictions.append(self.make_prediction(sample, self.tree, 1.0))
        return predictions

    def entropy(self, attribute_column):
        # find unique values and their frequency counts for the given attribute
        values, counts = np.unique(attribute_column, return_counts=True)

        # calculate entropy for each unique value
        entropy_list = []

        for i in range(len(values)):
          probability = counts[i]/np.sum(counts)
          entropy_list.append(-probability*np.log2(probability))

        # calculate sum of individual entropy values
        total_entropy = np.sum(entropy_list)

        return total_entropy
    
    def information_gain(self, data, feature_attribute_name, target_attribute_name):
        # find total entropy of given subset
        total_entropy = self.entropy(data[target_attribute_name])

        # find unique values and their frequency counts for the attribute to be split
        values, counts = np.unique(data[feature_attribute_name], return_counts=True)

        # calculate weighted entropy of subset
        weighted_entropy_list = []

        for i in range(len(values)):
          subset_probability = counts[i]/np.sum(counts)
          subset_entropy = self.entropy(data.where(data[feature_attribute_name]==values[i]).dropna()[target_attribute_name])
          weighted_entropy_list.append(subset_probability*subset_entropy)

        total_weighted_entropy = np.sum(weighted_entropy_list)

        # calculate information gain
        information_gain = total_entropy - total_weighted_entropy

        return information_gain
    
    def decision_tree(self, data, orginal_data, feature_attribute_names, target_attribute_name,
                      Print=False, depth="", parent_node_class=None):
        # base cases:
        # if data is pure, return the majority class of subset
        unique_classes = np.unique(data[target_attribute_name])
        if len(unique_classes) <= 1:
            if(Print):
                print(f"{depth}prediction = {unique_classes[0].decode()}")
            return unique_classes[0]
        # if subset is empty, ie. no samples, return majority class of original data
        elif len(data) == 0:
          majority_class_index = np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])
          return np.unique(original_data[target_attribute_name])[majority_class_index]
        # if data set contains no features to train with, return parent node class
        elif len(feature_attribute_names) == 0:
          return parent_node_class
        # if none of the above are true, construct a branch:
        else:
          # determine parent node class of current branch
          majority_class_index = np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])
          parent_node_class = unique_classes[majority_class_index]

          # determine information gain values for each feature
          # choose feature which best splits the data, ie. highest value
          ig_values = [self.information_gain(data, feature, target_attribute_name) for feature in feature_attribute_names]
          best_feature_index = np.argmax(ig_values)
          self.ig_splits.append(ig_values[best_feature_index])
          best_feature = feature_attribute_names[best_feature_index]
            
          # create tree structure, empty at first
          tree = {best_feature: {}}

          # remove best feature from available features, it will become the parent node
          feature_attribute_names = [i for i in feature_attribute_names if i != best_feature]

          # create nodes under parent node
          parent_attribute_values = np.unique(data[best_feature])
          for value in parent_attribute_values:
            sub_data = data.where(data[best_feature] == value).dropna()
            if Print:
                print(f"{depth}{best_feature} = {value.decode()}")
            # call the algorithm recursively
            subtree = self.decision_tree(sub_data, orginal_data, feature_attribute_names,
                                         target_attribute_name, Print, depth+'    ', parent_node_class)

            # add subtree to original tree
            tree[best_feature][value] = subtree

          return tree
    
    def make_prediction(self, sample, tree, default=1):
        # map sample data to tree
        for attribute in list(sample.keys()):
          # check if feature exists in tree
          if attribute in list(tree.keys()):
            try:
                result = tree[attribute][sample[attribute]]
            except:
                return default

            result = tree[attribute][sample[attribute]]

            # if more attributes exist within result, recursively find best result
            if isinstance(result, dict):
                return self.make_prediction(sample, result)
            else:
                return result
    
    def score(self, X, y):
        """ Return accuracy(Classification Acc) of model on a given dataset. Must implement own score function.

        Args:
            X (array-like): A 2D numpy array with data, excluding targets
            y (array-like): A 1D numpy array of the targets 
        """
        pred = self.predict(X)
        y = np.array(self.convert(list(y)))
        return np.mean(pred == y)
    
    def convert(self, data):
        if isinstance(data, bytes):  return data.decode()
        if isinstance(data, dict):   return dict(map(self.convert, data.items()))
        if isinstance(data, tuple):  return tuple(map(self.convert, data))
        if isinstance(data, list):   return list(map(self.convert, data))
        return data
    





## 1.1 Debug 

Debug your model by training on the lenses dataset: [Debug Dataset](https://github.com/cs472ta/CS472/blob/master/datasets/lenses.arff)

Test your model on the lenses test set: [Debug Test Dataset](https://github.com/cs472ta/CS472/blob/master/datasets/lenses_test.arff)

Parameters:
validation_size = 0.0

---

Expected Results: Accuracy = [0.33]

Predictions should match this file: [Lenses Predictions](https://github.com/cs472ta/CS472/blob/master/debug_solutions/pred_lenses.csv)

<!-- You should be able to get about 68% (61%-82%) predictive accuracy on the lenses data -->

In [25]:
# Load debug training data
data = pd.DataFrame(arff.loadarff('datasets/lenses.arff')[0], dtype=float)

# Train Decision Tree
x_train = data.drop(columns="contact_lenses")
y_train = data["contact_lenses"]
dt = DTClassifier()
dt.fit(x_train, y_train, True)
print(dt.ig_splits)

# Load debug test data
data = pd.DataFrame(arff.loadarff('datasets/lenses_test.arff')[0], dtype=float)

# Predict and compute model accuracy
x_test = data.drop(columns="contact_lenses")
y_test = data["contact_lenses"]
dt.score(x_test, y_test)


tear_prod_rate = normal
    astigmatism = no
        age = pre_presbyopic
            prediction = soft
        age = presbyopic
            spectacle_prescrip = hypermetrope
                prediction = soft
            spectacle_prescrip = myope
                prediction = none
        age = young
            prediction = soft
    astigmatism = yes
        spectacle_prescrip = hypermetrope
            age = pre_presbyopic
                prediction = none
            age = presbyopic
                prediction = none
            age = young
                prediction = hard
        spectacle_prescrip = myope
            prediction = hard
tear_prod_rate = reduced
    prediction = none
[0.5487949406953985, 0.7704260414863778, 0.3166890883150208, 1.0, 0.4591479170272448, 0.9182958340544896]


0.3333333333333333

In [ ]:
# Optional/Additional Debugging Dataset - Pizza Homework
# pizza_dataset = np.array([[1,2,0],[0,0,0],[0,1,1],[1,1,1],[1,0,0],[1,0,1],[0,2,1],[1,0,0],[0,2,0]])
# pizza_labels = np.array([2,0,1,2,1,2,1,1,0])

## 1.2 Evaluation

We will evaluate your model based on its performance on the zoo dataset. 

Train your model using this dataset: [Evaluation Train Dataset](https://github.com/cs472ta/CS472/blob/master/datasets/zoo.arff)

Test your model on this dataset: [Evaluation Test Dataset](https://github.com/cs472ta/CS472/blob/master/datasets/zoo_test.arff)

Parameters: 
validation_size = 0.0

---
Save predictions as a .csv and which you'll submit together with this notebook on LearningSuite **


In [26]:
# Load evaluation training data
data = pd.DataFrame(arff.loadarff('datasets/zoo.arff')[0])

# Train Decision Tree
x_train = data.drop(columns="type")
y_train = data["type"]
dt = DTClassifier()
dt.fit(x_train, y_train, True)
print(dt.ig_splits)

# Load evaluation test data
data = pd.DataFrame(arff.loadarff('datasets/zoo_test.arff')[0])

# Predict and compute model accuracy
x_test = data.drop(columns="type")
y_test = data["type"]
dt.score(x_test, y_test)


legs = 0
    fins = F
        toothed = F
            prediction = c7
        toothed = T
            prediction = c3
    fins = T
        eggs = F
            prediction = cT
        eggs = T
            prediction = c4
legs = 2
    hair = F
        prediction = c2
    hair = T
        prediction = cT
legs = 4
    hair = F
        predator = F
            prediction = c3
        predator = T
            toothed = F
                prediction = c7
            toothed = T
                prediction = c5
    hair = T
        prediction = cT
legs = 5
    prediction = c7
legs = 6
    predator = F
        prediction = c6
    predator = T
        prediction = c7
legs = 8
    prediction = c7
[1.3630469031539394, 0.8865408928220899, 0.9852281360342515, 0.6962122601251458, 0.8256265261578954, 0.6892019851173654, 0.8631205685666308, 0.7219280948873623, 0.7219280948873623]


0.147

## 2. (20%) You will use your ID3 algorithm to induce decision trees for the cars dataset and the voting dataset.  Do not use a stopping criteria, but induce the tree as far as it can go (until classes are pure or there are no more data or attributes to split on).  
- Implement and use 10-fold CV on each data set to predict how well the models will do on novel data.  
- For each dataset, report the training and test classification accuracy for each fold and the average test accuracy. 
- Summarize these results, and discuss what you observed. Note that with a full tree you will often get 100% accuracy on the training set. Why would you and in what cases would you not?  
- As a rough sanity check, typical decision tree accuracies for these data sets are: Cars: .90-.95, Vote: .92-.95.

## 2.1 Implement 10-fold Cross Validation

In [239]:
def fold_i_of_k(dataset, i, k):
    n = len(dataset)
    return dataset[n*(i-1)//k:n*i//k]

def k_fold_cross_validate(data, output_class, folds=3):
    data = data.sample(frac=1).reset_index(drop=True)
    data_set = []
    scores = []
    j = 0
    for i in range(1, folds+1):
        data_set.append(fold_i_of_k(data, i, folds))
    for i in range(folds):
        dfs_1 = data_set[0:i]
        dfs_2 = data_set[i+1:len(data_set)+1]
        df = dfs_1 + dfs_2
        df = pd.concat(df)
        x_train = df.drop(columns=output_class)
        y_train = df[output_class]
        x_test = data_set[i].drop(columns=output_class)
        y_test = data_set[i][output_class]
        dt = DTClassifier()
        dt.fit(x_train, y_train)
        score = dt.score(x_test, y_test)
        print(score)
        scores.append(score)
    return scores

##  2.2 Cars Dataset
- Use this [Cars Dataset](https://github.com/cs472ta/CS472/blob/master/datasets/cars.arff)

In [240]:
# Use 10-fold CV on Cars Dataset
data = arff.loadarff('datasets/cars.arff')[0]
df =  pd.DataFrame(data)
scores = k_fold_cross_validate(df, "class", 10)
print(f"average accuracy: {np.mean(np.mean(scores))}")
cdt = dt.tree
# Report Average Test Accuracy

0.8953488372093024
0.9190751445086706
0.884393063583815
0.861271676300578
0.930635838150289
0.9069767441860465
0.8901734104046243
0.8728323699421965
0.884393063583815
0.9190751445086706
average accuracy: 0.8964175292378009


Discuss your results

## 2.3 Voting Dataset
- Use this [Voting Dataset with missing values](https://github.com/cs472ta/CS472/blob/master/datasets/voting_with_missing.arff)
- Note that you will need to support unknown attributes in the voting data set. 

In [241]:
# Used 10-fold CV on Voting Dataset
data = arff.loadarff('datasets/voting_with_missing.arff')[0]
df =  pd.DataFrame(data)
df = df.replace("?", "unknown")
scores = k_fold_cross_validate(df, "Class", 10)
print(f"average accuracy: {np.mean(np.mean(scores))}")
# Report Training and Test Classification Accuracies
# Report Average Test Accuracy

0.9302325581395349
0.9772727272727273
1.0
0.9545454545454546
0.9534883720930233
0.9318181818181818
0.9534883720930233
0.9318181818181818
0.8604651162790697
0.9318181818181818
average accuracy: 0.9424947145877379


Discuss your results

## 3. (10%) For each of the two problems above, summarize in English what the decision tree has learned (i.e. look at the induced tree and describe what rules it has discovered to try to solve each task). 
- If the tree is very large you can just discuss a few of the more shallow attributes combinations and the most important decisions made high in the tree.
- To visualize your trees, you could use something like sklearn's export_graphviz

## 3.1 Cars Decision Tree Summary

Discuss what the decision tree induced on the cars dataset has learned

In [242]:
safety, persons, buying, maint

NameError: name 'safety' is not defined

## 3.2 Voting Decision Tree Summary

Discuss what the decision tree induced on the voting dataset has learned

In [ ]:
physician-fee-freeze = ?
    education-spending = ?
physician-fee-freeze = n
    adoption-of-the-budget-resolution = ?
physician-fee-freeze = y
    synfuels-corporation-cutback = ?

## 4.  (5%) How did you handle unknown attributes in the voting problem? Why did you choose this approach? (Do not use the approach of just throwing out data with unknown attributes).

 I made it a thrid option

## 5.1 (10%) Use SciKit Learn's decision tree on the cars and voting datasets and compare your results. Try some of the approaches available to avoid overfit and report if that does better on a test set. 

### 5.1.1 SK Learn on Cars Dataset
- Use this [Cars Dataset](https://github.com/cs472ta/CS472/blob/master/datasets/cars.arff)

In [272]:
# Use SK Learn's Decision Tree to learn the cars dataset
def convert( data):
    if isinstance(data, bytes):  return data.decode()
    if isinstance(data, dict):   return dict(map(convert, data.items()))
    if isinstance(data, tuple):  return tuple(map(convert, data))
    if isinstance(data, list):   return list(map(convert, data))
    return data
data = arff.loadarff('datasets/cars.arff')[0]
data = convert(data)
df =  pd.DataFrame(data, dtype=float)


clf = tree.DecisionTreeClassifier()
X = df.drop(columns="class").values
y = df["class"].values
X = np.array(convert(X.tolist()))
y = np.array(convert(y.tolist()))

clf = clf.fit(X, y)

# Explore methods to avoid overfit

ValueError: could not convert string to float: 'vhigh'

Discuss results & compare to your method's results

### 5.1.2 SK Learn on Voting Dataset
- Use this [Voting Dataset with missing values](https://github.com/cs472ta/CS472/blob/master/datasets/voting_with_missing.arff)

In [2]:
# Use SK Learn's Decision Tree to learn the voting dataset

# Explore methods to avoid overfit

Discuss results & compare to your method's results

## 5.2 (10%) Choose a data set of your choice (not already used in this or previous labs) and use the SK decision tree to learn it. Experinent with different hyper-parameters to try to get the best results possible.

In [273]:
iris = load_iris()
X = iris['data']
y = iris['target']
X_train, X_test, y_train, y_test = tts(X,y)

depths = [1, 3, 4, 5, 6,7, 8, 9, 10, 11, 12]
max_feats = [2,'log2','sqrt','auto']
crit = ['gini', 'entropy']
scores = np.zeros((5,4,2))
maximum, argm = 0, []
for i,j,k in product(range(5),range(4),range(2)):
    d,m,c = depths[i], max_feats[j], crit[k]
    t = tree.DecisionTreeClassifier(criterion=c, max_depth=d, max_features=m)
    t.fit(X_train,y_train)
    new_score = t.score(X_test,y_test)
    if new_score > maximum:
        argm = [i,j,k]
        maximum = new_score
    scores[i][j][k] = new_score

print(f'Best score: {round(maximum, 5)*100} at depth: {depths[argm[0]]}, max_features: {max_feats[argm[1]]}, and criterion: {crit[argm[2]]}')

NameError: name 'load_iris' is not defined

## 6. (5%) Visualize your decision tree for your chosen data set (using export_graphviz or another tool) and discuss what you find. If your tree is too deep to reasonably fit on one page, show only the first several levels (e.g. top 5).

In [ ]:
# Include decision tree visualization here
t = tree.DecisionTreeClassifier(max_depth=depths[argm[0]],
                                max_features=max_feats[argm[1]],
                                criterion=crit[argm[2]])
t.fit(X_train, y_train)
ofile = tree.export_graphviz(t)
plt.figure(figsize=(20,20))
text = tree.plot_tree(t)
# Discuss what the model has learned

## 7. (optional 5% extra credit) Implement reduced error pruning to help avoid overfitting.  
- You will need to take a validation set out of your training data to do this, while still having a test set to test your final accuracy. 
- Create a table comparing the original trees created for your ID3 version with the cars and voting data sets with no overfit avoidance in part 2 and the new trees you create with pruning. 
- This table should compare:
    - a) The # of nodes (including leaf nodes) and tree depth of the final decision trees 
    - b) The generalization (test set) accuracy. (For the unpruned 10-fold CV models, just use their average values in the table).